In [2]:
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

In [4]:
from rpy2.robjects.packages import importr

utils = importr("utils")
utils.chooseCRANmirror(ind=1)
utils.install_packages('lme4')

%load_ext rpy2.ipython

R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/lme4_1.1-27.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 3311365 bytes (3.2 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to con

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [19]:
# Load data
conversations = pd.read_csv("results/intelligibility/conversations.csv")
conversations_melted = pd.read_csv("results/intelligibility/conversations_melted.csv")

# convert True/False to 0/1:
conversations.replace({False: 0, True: 1}, inplace=True)
conversations_melted.replace({False: 0, True: 1}, inplace=True)

conversations.head()


,utterance_id,speaker_code,tokens,pos,age,corpus,transcript_file,child_name,speaker_code_next,start_time_next,...,follow_up_start_time,follow_up_end_time,follow_up_is_speech_related,follow_up_is_intelligible,follow_up_speech_act,response_latency,response_latency_follow_up,has_response,response_is_clarification_request,pos_feedback
0,28,CHI,"[""let's"", 'get', 'some', 'glue', 'too', '.']","['v', 'pro:obj', 'v', 'qn', 'n', 'adv']",30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,96687.0,...,97154.0,100026.0,1,1,PF,0.0,467.0,1,0,1
1,50,CHI,"['i', 'get', 'some', 'glue', 'too', '.']","['pro:sub', 'v', 'qn', 'n', 'adv']",30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,148262.0,...,150114.0,151434.0,1,1,AA,0.0,1852.0,1,0,1
2,52,CHI,"['broken', '.']",['part'],30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,151434.0,...,155422.0,161449.0,1,1,ST,0.0,3988.0,1,0,1
3,58,CHI,"['huh', 'right', 'there', 'see', 'his', 'legs'...","['co', 'adv', 'adv', 'v', 'det:poss', 'n', 'v'...",30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,169064.0,...,172202.0,173521.0,1,1,AD,0.0,3138.0,1,0,1
4,78,CHI,"['hi', 'jenny', 'how', 'you', 'fine', 'hi', 'h...","['co', 'n:prop', 'pro:rel', 'pro:per', 'adv', ...",30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,243831.0,...,249125.0,249641.0,1,1,AA,0.0,5294.0,1,0,1


In [20]:
# Some children have very few data points:
counts = conversations.groupby("corpus").size()
counts

corpus
Bloom                   1042
Braunwald               1948
Brent                   2076
Forrester                205
Gleason                 6628
MPI-EVA-Manchester    139924
MacWhinney              9672
NewmanRatner             140
Peters                  5363
Providence             46594
Rollins                  113
Sachs                   2596
Snow                    6105
Soderstrom              1404
Thomas                111165
Tommerdahl              5287
Weist                   8395
dtype: int64

In [21]:
# normalize age
min_age, max_age = conversations["age"].min(), conversations["age"].max()
conversations["age"] = (conversations["age"] - min_age) / (max_age - min_age) * (1 - 0)

## Quality of communicative feedback/ Caregiver contingency


### Timing:

In [22]:
%%R -i conversations
library(lme4)

m_caregiver_contingency<-glmer('has_response ~ utt_is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: has_response ~ utt_is_intelligible * age + (1 | child_name)
   Data: conversations

      AIC       BIC    logLik  deviance  df.resid 
 292958.1  293012.0 -146474.1  292948.1    348652 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-24.3846   0.0935   0.4958   0.5367   3.9572 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 3.596    1.896   
Number of obs: 348657, groups:  child_name, 52

Fixed effects:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)              2.33381    0.14693  15.884   <2e-16 ***
utt_is_intelligible      0.81883    0.04350  18.822   <2e-16 ***
age                      0.95097    0.07368  12.906   <2e-16 ***
utt_is_intelligible:age  0.05398    0.07930   0.681    0.496    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation o

### Clarification requests

In [ ]:
%%R -i conversations
library(lme4)

# TODO: filter out no response cases?
m_caregiver_contingency<-glmer('response_is_clarification_request ~ utt_is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))

### Combined (Positive Feedback = No pause, no clarification request)

In [ ]:
%%R -i conversations
library(lme4)

m_caregiver_contingency<-glmer('pos_feedback ~ utt_is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))

## Effect of communicative feedback
### Positive Feedback: Timing

In [ ]:
%%R -i conversations
library(lme4)

# conversations_child_intelligible = subset(conversations, is_intelligible==1)
# TODO: only intelligible ones?

m_child_contingency<-glmer('follow_up_is_intelligible ~ has_response * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_child_contingency))

### Negative Feedback: Clarification requests

In [ ]:
%%R -i conversations_melted
library(lme4)

conversations_melted_cr = subset(conversations_melted, response_is_clarification_request == 1)


m_child_contingency<-glmer('is_intelligible ~ is_follow_up + (1 | age) + (1 | child_name) + (1 | is_follow_up)', data=conversations_melted_cr, family=binomial)
print(summary(m_child_contingency))

In [ ]:
%%R -i conversations_melted
library(lme4)

m_child_contingency<-glmer('is_intelligible ~ response_is_clarification_request * is_follow_up + (1 | age) + (1 | child_name) + (1 | is_follow_up)', data=conversations_melted, family=binomial)
print(summary(m_child_contingency))